In [1]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.distributions.empirical_distribution import ECDF
from datetime import datetime as dt


warnings.filterwarnings("ignore")
%matplotlib inline

In [7]:
# Train Dataset
train_df = pd.read_csv("./data/train.csv", parse_dates=True, low_memory=False, index_col='Date')

# Store Data
store_df = pd.read_csv("./data/test.csv", low_memory=False)

# Test Dataset
test_df = pd.read_csv("./data/test.csv", parse_dates=True, low_memory=False, index_col='Date')

# Sample Submission
sample_df = pd.read_csv("./data/sample_submission.csv")


In [8]:
train_df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1
2015-07-31,2,5,6064,625,1,1,0,1
2015-07-31,3,5,8314,821,1,1,0,1
2015-07-31,4,5,13995,1498,1,1,0,1
2015-07-31,5,5,4822,559,1,1,0,1


In [9]:
test_df.head()

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday
Date,,,,,,,
2015-09-17,1,1,4,1.0,1,0,0
2015-09-17,2,3,4,1.0,1,0,0
2015-09-17,3,7,4,1.0,1,0,0
2015-09-17,4,8,4,1.0,1,0,0
2015-09-17,5,9,4,1.0,1,0,0
